In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import json
from haversine import haversine
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))
os.getcwd()

'/home/jian/Projects/Smoothie_King/adjust_for_pending_stores'

In [2]:
writer_folder="./output_"+str(datetime.datetime.now().date())+"/"
try:
    os.stat(writer_folder)
except:
    os.mkdir(writer_folder)

In [3]:
excel_output=pd.ExcelFile("/home/jian/Projects/Smoothie_King/weekly_TA_updates/output_20191202/SmoothieKing_TA_of_3_miles_zips_JL_2019-12-03.xlsx")
excel_output.sheet_names


['summary_by_TA', 'output_TA_by_store', 'zip_TA', 'summary_by_store_count']

In [4]:
df_by_store=excel_output.parse("output_TA_by_store",dtype=str)
df_by_store.shape

(957, 18)

In [5]:
new_store_excel=pd.ExcelFile("/home/jian/Projects/Smoothie_King/SK_celery/force_store_list/SK_forced_store_list_20191204.xlsx")
print(new_store_excel.sheet_names)
df_new_stores=new_store_excel.parse("opening",dtype=str)
new_store_list=df_new_stores['storenumber'].unique().tolist()
print(len(new_store_list))
print(new_store_list)

['opening', 'closed']
6
['1797', '1808', '1768', '1671', '1584', '1709']


In [6]:
df_by_store[df_by_store['storenumber'].isin(new_store_list)]

# All of the new stores need to be added

,storenumber,Address,city,state,zip,StoreOpenDate,Latitude,Longitude,Ownership,Franchisee_Name,IPAddress,DriveThru,status,StoreCloseDate,used_lat,used_long,zips_3_miles,TA
903,1671,8361 W. Sunrise Blvd,Sunrise,FL,33322,2019-11-14 00:00:00,26.148596,-80.262615,Franchised,Millie Medrano,NaN,N,New,NaN,26.148596,-80.262615,"['33084', '33388', '33336', '33324', '33359', ...","4_['0510', '0587', '1139', '1671']"
938,1768,1100 W Main St,Lewisville,TX,75067,2019-11-07 00:00:00,33.0417436,-97.0227896,Franchised,Jackie Ben-Meir,NaN,Y,New,NaN,33.0417436,-97.0227896,"['75027', '75028', '75067', '75029', '75057']","3_['0386', '0547', '1768']"
949,1797,"1659 STATE HIGHWAY 46 W, SUITE 175",New Braunfels,TX,78132,2019-11-04 00:00:00,29.713163,-98.160644,Franchised,Paul Flores,NaN,N,New,NaN,29.713163,-98.160644,"['78132', '78131']",1_['1797']


In [7]:
df_new_stores=df_new_stores[~df_new_stores['storenumber'].isin(df_by_store['storenumber'].tolist())]
df_new_stores

,storenumber
1,1808
4,1584
5,1709


In [8]:
TA_name=df_by_store[['storenumber','TA']].drop_duplicates()
TA_name_count=TA_name.groupby("TA")['storenumber'].count().to_frame().reset_index().rename(columns={"storenumber":"store_count"})
TA_name_list=TA_name.groupby("TA")['storenumber'].apply(list).to_frame().reset_index().rename(columns={"storenumber":"store_list"})
TA_name=pd.merge(TA_name_count,TA_name_list,on="TA",how="outer")
TA_name['store_count']=TA_name['store_count'].apply(lambda x: str(x))
TA_name['store_list']=TA_name['store_list'].apply(lambda x: str(x))
TA_name['TA_new_name']=TA_name['store_count']+"_"+TA_name['store_list']
TA_name_dict=TA_name.set_index("TA").to_dict()['TA_new_name']


df_by_store['TA']=df_by_store['TA'].apply(lambda x: TA_name_dict[x])
df_by_store=df_by_store.rename(columns={"TA":"TA_new"})


In [9]:
zip_TA=excel_output.parse("zip_TA",dtype=str)
zip_TA['TA']=zip_TA['TA'].apply(lambda x: TA_name_dict[x])
zip_TA=zip_TA.rename(columns={"TA":"TA_new"})

# The automatic run region is not updated
del zip_TA['Co-Op/Regional']

In [10]:
region_by_store_file=pd.ExcelFile("/home/jian/Projects/Smoothie_King/adjust_for_pending_stores/region_mapping/Updated RAF List - Number of Opened Stores-2019-09-13.xlsx")
mapping_store_region=region_by_store_file.parse('All Stores',dtype=str)
mapping_store_region=mapping_store_region[['Store: Store Number  ↑','Co-Op/Regional  ↑']]
mapping_store_region.columns=['store','Co-Op/Regional']
mapping_store_region['Co-Op/Regional']=mapping_store_region['Co-Op/Regional'].fillna("nan")
mapping_store_region['store']=mapping_store_region['store'].apply(lambda x: x.replace("SK-",""))
print(mapping_store_region['store'].apply(lambda x: len(x)).unique())

zip_TA=pd.merge(zip_TA,mapping_store_region,on="store",how="left")
zip_TA['Co-Op/Regional']=zip_TA['Co-Op/Regional'].fillna("Unknown")

[4]


In [11]:
df_new_stores

,storenumber
1,1808
4,1584
5,1709


In [12]:
dict_new_stores={}

# 1808	MSY Airport Metarie, LA 70062
dict_new_stores.update({"1808":{"GoogleAddress":"1 Terminal Dr, Kenner, LA 70062","Google_LatLng":[29.9910905,-90.2614186]}})

# 1584	1030 Ave D Westwego LA, 70094
dict_new_stores.update({"1584":{"GoogleAddress":"1030 Avenue D, Westwego, LA 70094","Google_LatLng":[29.8991779,-90.1479471]}})

# 1709	2525 Jackson Rd Ann Arbor MI, 48105
dict_new_stores.update({"1709":{"GoogleAddress":"2635 Jackson Ave, Ann Arbor, MI 48103","Google_LatLng":[42.2808521,-83.7832389]}})



In [13]:
df_new_stores['GoogleAddress']=df_new_stores['storenumber'].apply(lambda x: dict_new_stores[x]['GoogleAddress'])
df_new_stores['GoogleLatitude']=df_new_stores['storenumber'].apply(lambda x: dict_new_stores[x]['Google_LatLng'][0])
df_new_stores['GoogleLongitude']=df_new_stores['storenumber'].apply(lambda x: dict_new_stores[x]['Google_LatLng'][1])


In [14]:
df_new_stores['zips_3_miles']=np.nan

for ind,row in df_new_stores.iterrows():
    store_center=(row['GoogleLatitude'],row['GoogleLongitude'])
    zip_list=[]
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],unit="mi")
        if dist<=3:
            zip_list.extend([zip_cd])
    df_new_stores.loc[ind,'zips_3_miles']=str(zip_list)


In [15]:
df_new_stores

,storenumber,GoogleAddress,GoogleLatitude,GoogleLongitude,zips_3_miles
1,1808,"1 Terminal Dr, Kenner, LA 70062",29.991090,-90.261419,"['70065', '70062', '70063', '70003', '70141']"
4,1584,"1030 Avenue D, Westwego, LA 70094",29.899178,-90.147947,"['70073', '70094']"
5,1709,"2635 Jackson Ave, Ann Arbor, MI 48103",42.280852,-83.783239,"['48106', '48103', '48104', '48107', '48109']"


In [16]:
cols=zip_TA.columns.tolist()
zip_ta_new_stores=pd.DataFrame(columns=zip_TA.columns.tolist())
for ind, row in df_new_stores.iterrows():
    store=row['storenumber']
    zip_list=eval(row['zips_3_miles'])
    df=pd.DataFrame({"zip":zip_list},index=[store]*len(zip_list)).reset_index().rename(columns={"index":"store"})
    df['TA_new']="new_stores"
    df['Primary_DMA']="new_stores"
    df['Co-Op/Regional']="Unknown"
    zip_ta_new_stores=zip_ta_new_stores.append(df)
zip_ta_new_stores=zip_ta_new_stores[cols]

/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [17]:
zip_TA=zip_TA.append(zip_ta_new_stores)
df_TA_zips=zip_TA.copy()

In [18]:
df_TA_zips[df_TA_zips['Co-Op/Regional']=="nan"]

,store,TA_new,zip,Primary_DMA,Co-Op/Regional
124,0016,1_['0016'],70605,LAKE CHARLES,nan
125,0016,1_['0016'],70609,LAKE CHARLES,nan
126,0016,1_['0016'],70606,LAKE CHARLES,nan
127,0016,1_['0016'],70607,LAKE CHARLES,nan
195,0032,"3_['0032', '1146', '1686']",70361,NEW ORLEANS,nan
...,...,...,...,...,...
6508,1812,1_['1812'],60585,CHICAGO,nan
6509,1841,"4_['0271', '0995', '1532', '1841']",75080,DALLAS-FT. WORTH,nan
6510,1841,"4_['0271', '0995', '1532', '1841']",75075,DALLAS-FT. WORTH,nan
6511,1841,"4_['0271', '0995', '1532', '1841']",75083,DALLAS-FT. WORTH,nan


In [19]:
for region,df_group in df_TA_zips.groupby("Co-Op/Regional"):
    if region!="UnknowRegion":
        df_group=df_group[['zip','Co-Op/Regional']].drop_duplicates()
        df_group.to_csv(writer_folder+"zips_for_the_%s.csv"%region,index=False)
    else:
        # df_austin=df_group[df_group['store'].isin(Austin_stores['store'].tolist())]
        df_unknown=df_group[~df_group['store'].isin(Austin_stores['store'].tolist())]

        # df_austin.to_csv(writer_folder+"zips_for_the_%s.csv"%"AustinDMA",index=False)
        df_unknown.to_csv(writer_folder+"zips_for_the_%s.csv"%"UnknownRegion",index=False)
df_output_overall_zips=df_TA_zips[['zip']].drop_duplicates()
df_output_overall_zips.to_csv(writer_folder+"overall_zips_in_TA.csv",index=False)

In [20]:
writer=pd.ExcelWriter(writer_folder+"SmoothieKing_TA_of_3_miles_zips_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_TA_zips.to_excel(writer,"df_TA_zips",index=False)
df_new_stores.to_excel(writer,"new_stores_add_on",index=False)
df_by_store.to_excel(writer,"df_by_store",index=False)
rfa_mapping=pd.read_excel("/home/jian/Projects/Smoothie_King/SK_celery/RFA_region_20190128.xlsx",dtype=str)
rfa_mapping.to_excel(writer,"RFA_region_20190128",index=False)
writer.save()

In [21]:
writer_folder

'./output_2019-12-04/'

In [22]:
sorted(df_TA_zips['Co-Op/Regional'].unique().tolist())

['Atlanta',
 'Austin',
 'Baton Rouge',
 'Biloxi',
 'Charlotte',
 'Cincinnati',
 'Dallas-Fort Worth',
 'Houston',
 'Jackson',
 'Kansas City',
 'Memphis',
 'Miami-Ft. Lauderdale',
 'Mobile-Pensacola-Destin',
 'Nashville',
 'New Orleans',
 'Orlando',
 'San Antonio',
 'St. Louis',
 'Tampa',
 'Unknown',
 'Washington DC',
 'nan']